## Patch and Organize data
In this file, I defined few functions to pretreat the data. I handled data related to height, but the functions I defined can be easily used to deal with other data without making change(or few change in some function). You are welcome to download this Jupyter file and test it with data in our size sheet.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import codecs
import csv
import re
import math

In [3]:
names_all = ['all_size','boy_size','girl_size','all_height','all_weight','boy_height',
         'boy_weight','girl_height','girl_weight','boy_top_long_chest',
         'boy_top_long_sleeve','boy_top_long_height','boy_top_long_sweep',
         'boy_top_long_undefined','boy_top_short_chest','boy_top_short_sleeve',
         'boy_top_short_height','boy_top_short_sweep','boy_bottom_pants_height',
         'boy_bottom_pants_crotch','boy_bottom_pants_waist','boy_bottom_pants_hip',
         'boy_bottom_pants_inseam','boy_bottom_pants_adjustable','boy_bottom_short_waist',
         'boy_bottom_short_hip','boy_bottom_short_inseam','boy_bottom_short_adjustable',
         'girl_top_long_chest','girl_top_long_sleeve','girl_top_long_height','girl_top_long_sweep',
         'girl_top_short_chest','girl_top_short_sleeve','girl_top_short_height',
         'girl_top_short_sweep','girl_bottom_long_waist','girl_bottom_long_hip','girl_bottom_long_inseam',
         'girl_bottom_long_adjustable','girl_bottom_short_waist','girl_bottom_short_hip',
         'girl_bottom_short_inseam','girl_bottom_short_adjustable','girl_dress_hollow_to_floor',
         'girl_dress_bust', 'girl_dress_waist','girl_dress_hips','girl_dress_sleeve',
         'girl_dress_inseam','girl_dress_height']

### Load file as pandas dataframe, set column index as names_all

In [4]:
file = pd.read_csv('data_v9.csv', names = names_all)

### show first 6 rows of data

In [5]:
file.head(6)

,all_size,boy_size,girl_size,all_height,all_weight,boy_height,boy_weight,girl_height,girl_weight,boy_top_long_chest,...,girl_bottom_short_hip,girl_bottom_short_inseam,girl_bottom_short_adjustable,girl_dress_hollow_to_floor,girl_dress_bust,girl_dress_waist,girl_dress_hips,girl_dress_sleeve,girl_dress_inseam,girl_dress_height
NaN,Sizes for all,Boy size,Girl size,All,NaN,Boy,NaN,Girl,NaN,boy Top (long sleeve),...,NaN,NaN,NaN,Girl Dress,NaN,NaN,NaN,NaN,NaN,NaN
Brand,NaN,NaN,NaN,Height,Weight,Height,Weight,Height,Weight,Chest,...,Hip,Inseam,Adjustable Y/N,Hollow to Floor,Bust,Waist,Hips,sleeve,Inseam,height
#sen,6-9mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#sen,9-12mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#sen,12-18mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#sen,18-24mo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### The first two rows are not well organized. Remove them, and add two more rows: boy_actual_height and girl_actual_height

In [20]:
file = file.iloc[2:]
file["boy_actual_height"] = np.nan
file["girl_actual_height"] = np.nan

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
inches = ['\"', 'inches', '\'']
cm = ['cm', 'CM']
kg = ['kg', 'KG']
lbs = ['lbs', 'LBS']

### Define checkHeightUnit function, which would return the unit of the data or None if the data is pure number.

In [7]:
def checkHeightUnit(data):
    unit = None
    for item in inches:
        if item in data:
            unit = 'inches'
            return unit
    for item in cm:
        if item in data:
            unit = 'cm'
            return unit
    return None

### Define column_organizer. This function will get the unit from data, (or keep the unit if the data is pure number) and uniform the format of data.

In [30]:
def column_organizer(column):
    unit = None
    for i in range(len(column)):
        if type(column[i]) is str or math.isnan(column[i]) is False:
            data = str(column[i])
            checkUnit = checkHeightUnit(data)
            if checkUnit is not None:
                unit = checkUnit
            nums = re.findall(r'\d+\.?\d+', data)
            if len(nums) is 1:
                column[i] = nums[0] + unit
            else:
                column[i] = nums[0] + '-' + nums[1] + unit               

### In order to simplify the process of analysis data, we need to complete the form using data in different columns. This patch_data is use to complete column using another.

In [40]:
def patch_data(column1, column2):
    for i in range(len(column1)):
        if type(column1[i]) is not str and math.isnan(column1[i]) is True:
            column1[i] = column2[i]

### Get data related to height

In [52]:
all_height = file['all_height']
boy_height = file['boy_height']
girl_height = file['girl_height']
boy_actual_height = file['boy_actual_height'].values
girl_actual_height = file['girl_actual_height'].values
boy_top_long_height = file['boy_top_long_height'].values
boy_top_short_height = file['boy_top_short_height'].values
boy_bottom_pants_height = file['boy_bottom_pants_height'].values
girl_top_long_height = file['girl_top_long_height'].values
girl_top_short_height = file['girl_top_short_height'].values
girl_dress_height = file['girl_dress_height'].values

### Add data to boy_height from all_height if boy_height is empty.
#### In the steps below, I used pandas dataframe object. However, reset data using dataframe object cost lots of time. I think it would be better if we using array to edit data, than transform array to dataframe. 

In [42]:
patch_data(boy_height, all_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Add data to boy_height from boy_top_long_height if boy_height is empty at the same row.

In [43]:
patch_data(boy_height, boy_top_long_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Add data to boy_height from boy_top_shor_height if boy_height is empty at the same row.

In [45]:
patch_data(boy_height, boy_top_short_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Using column_organizer to unify the unit in boy_height

In [48]:
column_organizer(boy_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


### Store data in boy_height to csv file

In [50]:
boy_height.to_csv('boy_height.csv')

### Below is the same process handling the girl_height

In [53]:
patch_data(girl_height, all_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
patch_data(girl_height, girl_top_long_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
patch_data(girl_height, girl_top_short_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
patch_data(girl_height, girl_dress_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
column_organizer(girl_height)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


### Copy the data to original table.

In [58]:
file['boy_height'] = boy_height
file['girl_height'] = girl_height

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Stored the table to half_parsed_v100.csv

In [59]:
file.to_csv('half_parsed_v100.csv')